In [6]:
# Install required libraries in Colab
!pip install yfinance pandas twilio requests

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
from twilio.rest import Client
import time

# Twilio credentials (replace with your own)
ACCOUNT_SID = "YOUR_TWILIO_ACCOUNT_SID"  # Replace with your Twilio SID
AUTH_TOKEN = "YOUR_TWILIO_AUTH_TOKEN"    # Replace with your Twilio Token
TWILIO_PHONE_NUMBER = "whatsapp:+14155238886"  # Twilio WhatsApp number
YOUR_WHATSAPP_NUMBER = "whatsapp:+85291644818"  # Your WhatsApp number (e.g., whatsapp:+12345678901)

# Initialize Twilio client
client = Client(ACCOUNT_SID, AUTH_TOKEN)

# Function to send WhatsApp notification
def send_whatsapp_notification(message):
    message = client.messages.create(
        body=message,
        from_=TWILIO_PHONE_NUMBER,
        to=YOUR_WHATSAPP_NUMBER
    )
    print(f"Notification sent: {message.sid}")

# Function to get S&P 500 tickers
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    return sp500_table['Symbol'].tolist()

# Function to gather financial calendar data
def get_financial_calendar(tickers, days_ahead=7):
    end_date = datetime.now() + timedelta(days=days_ahead)
    calendar_data = []

    for ticker in tickers[:10]:  # Limiting to 10 for testing; remove slice for all
        try:
            stock = yf.Ticker(ticker)
            # Get earnings dates
            earnings_dates = stock.calendar
            if earnings_dates is not None and not earnings_dates.empty:
                for date in earnings_dates.columns:
                    event_date = pd.to_datetime(earnings_dates[date].iloc[0])
                    if event_date and datetime.now() <= event_date <= end_date:
                        calendar_data.append({
                            "Ticker": ticker,
                            "Event": "Earnings",
                            "Date": event_date,
                            "Details": "Earnings Report"
                        })

            # Get dividend data
            dividends = stock.dividends
            if not dividends.empty:
                latest_dividend_date = dividends.index[-1]
                if datetime.now() <= latest_dividend_date <= end_date:
                    calendar_data.append({
                        "Ticker": ticker,
                        "Event": "Dividend",
                        "Date": latest_dividend_date,
                        "Details": f"Dividend: ${stock.info.get('dividendYield', 0) * stock.info.get('previousClose', 0):.2f}"
                    })

            # Get financials (revenue, earnings)
            financials = stock.financials
            if not financials.empty:
                revenue = financials.loc['Total Revenue'].iloc[0] / 1e9  # Convert to billions
                calendar_data.append({
                    "Ticker": ticker,
                    "Event": "Revenue",
                    "Date": datetime.now(),  # Latest reported
                    "Details": f"Revenue: ${revenue:.2f}B"
                })

            print(f"Processed {ticker}")
            time.sleep(1)  # Avoid overwhelming the API
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return pd.DataFrame(calendar_data)

# Function to highlight important events
def highlight_important_events(df):
    # Define criteria for "important" (e.g., earnings this week, high dividends)
    important_events = df[
        (df['Event'] == "Earnings") &
        (df['Date'] <= datetime.now() + timedelta(days=7))
    ].copy()

    # Add high dividend events (example: yield > 0.03)
    for _, row in df.iterrows():
        if row['Event'] == "Dividend" and "Dividend: $" in row['Details']:
            dividend_value = float(row['Details'].split('$')[1])
            if dividend_value > 0.03:  # Arbitrary threshold
                important_events = pd.concat([important_events, pd.DataFrame([row])], ignore_index=True)

    return important_events

# Main execution
def main():
    print("Fetching S&P 500 tickers...")
    tickers = get_sp500_tickers()
    print(f"Found {len(tickers)} tickers.")

    print("Gathering financial calendar data...")
    calendar_df = get_financial_calendar(tickers)
    if calendar_df.empty:
        print("No upcoming events found.")
        return

    # Display the calendar data in Colab
    print("\nFull Financial Calendar:")
    display(calendar_df)  # Use display() for nicer output in Colab

    print("Highlighting important events...")
    important_events = highlight_important_events(calendar_df)

    if not important_events.empty:
        print("\nImportant Events:")
        display(important_events)
        for _, event in important_events.iterrows():
            message = (
                f"Important Event Alert!\n"
                f"Ticker: {event['Ticker']}\n"
                f"Event: {event['Event']}\n"
                f"Date: {event['Date'].strftime('%Y-%m-%d')}\n"
                f"Details: {event['Details']}"
            )
            send_whatsapp_notification(message)
            time.sleep(2)  # Avoid rate limiting
    else:
        send_whatsapp_notification("No important events found for the next 7 days.")
        print("No important events found.")

    # Optional: Save to CSV and download in Colab
    calendar_df.to_csv("sp500_financial_calendar.csv", index=False)
    from google.colab import files
    files.download("sp500_financial_calendar.csv")

# Run the script
main()

Fetching S&P 500 tickers...
Found 503 tickers.
Gathering financial calendar data...
Error processing MMM: 'dict' object has no attribute 'empty'
Error processing AOS: 'dict' object has no attribute 'empty'
Error processing ABT: 'dict' object has no attribute 'empty'
Error processing ABBV: 'dict' object has no attribute 'empty'
Error processing ACN: 'dict' object has no attribute 'empty'
Error processing ADBE: 'dict' object has no attribute 'empty'
Error processing AMD: 'dict' object has no attribute 'empty'
Error processing AES: 'dict' object has no attribute 'empty'
Error processing AFL: 'dict' object has no attribute 'empty'
Error processing A: 'dict' object has no attribute 'empty'
No upcoming events found.
